# Chapter 27: Reshaping By Pivoting and Grouping

In [7]:
import pandas as pd
import numpy as np
import catboost as cb
import collections

url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
   '2020-jetbrains-python-survey.csv'
jb = pd.read_csv(url)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_11696\2686139537.py:8: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


In [8]:
def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i,typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = (missing
         .drop(columns=[col])
         .values
        )
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

In [9]:
def prep_for_ml(df):
    # remove pandas types
    return (df
     .assign(**{col:df[col].astype(float) 
               for col in df.select_dtypes('number')},
             **{col:df[col].astype(str).fillna('') 
               for col in df.select_dtypes(['object', 'category'])})
    )

In [10]:
def get_uniq_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    uniq_cols = []
    for cols in counter.values():
        if len(cols) == 1:
            uniq_cols.extend(cols)
    return uniq_cols

In [11]:
def tweak_jb(jb):
     uniq_cols = get_uniq_cols(jb)
     return (jb
         [uniq_cols]
         .rename(columns=lambda c: c.replace('.', '_'))
         .assign(age=lambda df_:df_.age.str.slice(0,2).astype(float)
                    .astype('Int64'),
                are_you_datascientist=lambda df_:df_
                    .are_you_datascientist
                    .replace({'Yes': True, 'No': False, np.nan: False}),
                company_size=lambda df_:df_.company_size.replace({
                    'Just me': 1, 'Not sure': np.nan,
                    'More than 5,000': 5000, '2–10': 2, '11–50':11,
                    '51–500': 51, '501–1,000':501,
                    '1,001–5,000':1001}).astype('Int64'),
                country_live=lambda df_:df_.country_live
                    .astype('category'),
                employment_status=lambda df_:df_.employment_status
                     .fillna('Other').astype('category'),
                is_python_main=lambda df_:df_.is_python_main
                     .astype('category'),
                team_size=lambda df_:df_.team_size
                    .str.split(r'-', n=1, expand=True)
                    .iloc[:,0].replace('More than 40 people', 41)
                    .where(df_.company_size!=1, 1).astype(float),
                years_of_coding=lambda df_:df_.years_of_coding
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python_years=lambda df_:df_.python_years
                    .replace('Less than 1 year', .5)
                    .str.extract(r'(\d+)').astype(float),
                python3_ver=lambda df_:df_.python3_version_most
                     .str.replace('_', '.').str.extract(r'(\d\.\d)')
                     .astype(float),
                use_python_most=lambda df_:df_.use_python_most
                     .fillna('Unknown')
               )
        .assign(team_size=lambda df_:predict_col(df_, 'team_size')
             .astype(int))
        .drop(columns=['python2_version_most'])
        .dropna()
    )    

In [12]:
jb2 = tweak_jb(jb)

Learning rate set to 0.5
0:	learn: 2.9695218	total: 175ms	remaining: 3.32s
1:	learn: 2.8766539	total: 207ms	remaining: 1.86s
2:	learn: 2.8387189	total: 237ms	remaining: 1.34s
3:	learn: 2.8028751	total: 267ms	remaining: 1.07s
4:	learn: 2.7899957	total: 293ms	remaining: 880ms
5:	learn: 2.7749439	total: 322ms	remaining: 751ms
6:	learn: 2.7719128	total: 349ms	remaining: 647ms
7:	learn: 2.7649792	total: 373ms	remaining: 560ms
8:	learn: 2.7649588	total: 398ms	remaining: 487ms
9:	learn: 2.7630617	total: 425ms	remaining: 425ms
10:	learn: 2.7625779	total: 452ms	remaining: 370ms
11:	learn: 2.7515902	total: 480ms	remaining: 320ms
12:	learn: 2.7513459	total: 507ms	remaining: 273ms
13:	learn: 2.7445634	total: 532ms	remaining: 228ms
14:	learn: 2.7443257	total: 556ms	remaining: 185ms
15:	learn: 2.7423142	total: 580ms	remaining: 145ms
16:	learn: 2.7419143	total: 605ms	remaining: 107ms
17:	learn: 2.7399387	total: 631ms	remaining: 70.1ms
18:	learn: 2.7384296	total: 658ms	remaining: 34.6ms
19:	learn: 2.7

In [13]:
jb2.head()

,age,are_you_datascientist,company_size,country_live,employment_status,first_learn_about_main_ide,how_often_use_main_ide,ide_main,is_python_main,job_team,main_purposes,missing_features_main_ide,nps_main_ide,python_years,python3_version_most,several_projects,team_size,use_python_most,years_of_coding,python3_ver
1,21,True,5000,India,Fully employed by a company / organization,School / University,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",8.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",2,Software prototyping,3.0,3.6
2,30,False,5000,United States,Fully employed by a company / organization,Friend / Colleague,Daily,Vim,Yes,Work on your own project(s) independently,Both for work and personal,"No, it has all the features I need",10.0,3.0,Python 3_6,"Yes, I work on one main and several side projects",4,DevOps / System administration / Writing autom...,3.0,3.6
10,21,False,51,Other country,Fully employed by a company / organization,School / University,Daily,IntelliJ IDEA,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,1.0,Python 3_8,"Yes, I work on one main and several side projects",2,Web development,1.0,3.8
11,21,True,51,United States,Fully employed by a company / organization,Online learning platform / Online course,Daily,PyCharm Community Edition,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",9.0,3.0,Python 3_9,"Yes, I work on many different projects",2,Data analysis,3.0,3.9
13,30,True,5000,Belgium,Fully employed by a company / organization,Social network,Daily,VS Code,Yes,Work in a team,Both for work and personal,"No, it has all the features I need",10.0,6.0,Python 3_7,"Yes, I work on many different projects",2,Data analysis,3.0,3.7


## 27.1 A Basic Example

We can do pivot table in three ways:
- ``.pivot_table``
- ``pd.crosstab``
- ``.groupby``

In a pivot table, we have four things
- What we want to group by? (index/row)
- What columns do we want? (columns)
- What values do we want to groupby? (values)
- How do we want to aggregate them? (aggfunc)

In [14]:
(jb2
 .pivot_table(index='country_live', columns='employment_status', 
              values='age', aggfunc='mean'))

employment_status,Fully employed by a company / organization,Partially employed by a company / organization,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Working student
country_live,,,,
Algeria,31.2,24.0,27.0,<NA>
Argentina,30.632184,30.333333,28.3,23.0
Armenia,22.071429,<NA>,24.0,<NA>
Australia,32.935622,28.0,42.36,24.125
Austria,31.619565,30.357143,35.0,25.5
...,...,...,...,...
United States,32.429163,27.5,39.324324,21.842697
Uruguay,27.0,30.25,30.0,<NA>
Uzbekistan,21.0,21.0,<NA>,<NA>


- We can also use the ``pd.crosstab`` function

In [16]:
pd.crosstab(index=jb2.country_live,
            columns=jb2.employment_status,
            values=jb2.age,
            aggfunc='mean')

employment_status,Fully employed by a company / organization,Partially employed by a company / organization,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Working student
country_live,,,,
Algeria,31.2,24.0,27.0,<NA>
Argentina,30.632184,30.333333,28.3,23.0
Armenia,22.071429,<NA>,24.0,<NA>
Australia,32.935622,28.0,42.36,24.125
Austria,31.619565,30.357143,35.0,25.5
...,...,...,...,...
United States,32.429163,27.5,39.324324,21.842697
Uruguay,27.0,30.25,30.0,<NA>
Uzbekistan,21.0,21.0,<NA>,<NA>


- We could also  do this with a ``.groupby`` method

In [18]:
(jb2
 .groupby(['country_live', 'employment_status'])
 .age
 .mean()
 .unstack())

employment_status,Freelancer (a person pursuing a profession without a long-term commitment to any one employer),Fully employed by a company / organization,Other,Partially employed by a company / organization,Retired,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Student,Working student
country_live,,,,,,,,
Algeria,<NA>,31.2,<NA>,24.0,<NA>,27.0,<NA>,<NA>
Argentina,<NA>,30.632184,<NA>,30.333333,<NA>,28.3,<NA>,23.0
Armenia,<NA>,22.071429,<NA>,<NA>,<NA>,24.0,<NA>,<NA>
Australia,<NA>,32.935622,<NA>,28.0,<NA>,42.36,<NA>,24.125
Austria,<NA>,31.619565,<NA>,30.357143,<NA>,35.0,<NA>,25.5
...,...,...,...,...,...,...,...,...
United States,<NA>,32.429163,<NA>,27.5,<NA>,39.324324,<NA>,21.842697
Uruguay,<NA>,27.0,<NA>,30.25,<NA>,30.0,<NA>,<NA>
Uzbekistan,<NA>,21.0,<NA>,21.0,<NA>,<NA>,<NA>,<NA>


## 27.2 Using a Custom Aggregation Function

- Example: What is the percentage of Emacs users by country?

In [20]:
def per_emacs(ser):
    return ser.str.contains('Emacs').mean() * 100

In [25]:
# using pivot method
(jb2
 .pivot_table(index='country_live',
              values='ide_main',
              aggfunc= per_emacs))

,ide_main
country_live,
Algeria,0.000000
Argentina,3.669725
Armenia,0.000000
Australia,3.649635
Austria,1.562500
...,...
United States,4.486466
Uruguay,0.000000
Uzbekistan,0.000000


In [27]:
# using groupby
(jb2
 .groupby('country_live')
 [['ide_main']]
 .agg(per_emacs)
 )

,ide_main
country_live,
Algeria,0.000000
Argentina,3.669725
Armenia,0.000000
Australia,3.649635
Austria,1.562500
...,...
United States,4.486466
Uruguay,0.000000
Uzbekistan,0.000000


## 27.3 Multiple Aggregations

- What is the minimum and maximum age for each country?

In [28]:
# using pivot table method 
(jb2
 .pivot_table(index='country_live',
              values='age',
              aggfunc=(min, max))
)

,max,min
country_live,,
Algeria,60,18
Argentina,60,18
Armenia,30,18
Australia,60,18
Austria,50,18
...,...,...
United States,60,18
Uruguay,40,21
Uzbekistan,21,21


In [29]:
# using groupby method
(jb2
 .groupby('country_live')
 .age
 .agg([min, max]))

,min,max
country_live,,
Algeria,18,60
Argentina,18,60
Armenia,18,30
Australia,18,60
Austria,18,50
...,...,...
United States,18,60
Uruguay,21,40
Uzbekistan,21,21


## 27.4 Per Column Aggregations

In [31]:
(jb2
 .pivot_table(index='country_live',
              aggfunc=(min, max))
 .head()
)

C:\Users\Joanna\AppData\Local\Temp\ipykernel_11696\3467763400.py:1: FutureWarning: ['are_you_datascientist', 'employment_status', 'is_python_main'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  (jb2


age     company_size           first_learn_about_main_ide  \
             max min          max min                              max   
country_live                                                             
Algeria       60  18         5000   1  Technical review / Forum / Blog   
Argentina     60  18         5000   1  Technical review / Forum / Blog   
Armenia       30  18         5000   1  Technical review / Forum / Blog   
Australia     60  18         5000   1  Technical review / Forum / Blog   
Austria       50  18         5000   1  Technical review / Forum / Blog   

                                                         \
                                                    min   
country_live                                              
Algeria        Online learning platform / Online course   
Argentina      Online learning platform / Online course   
Armenia        Online learning platform / Online course   
Australia      Online learning platform / Online course   
Austria        Online learning platform / Online course   

             how_often_use_main_ide        ide_main        ... python_years  \
                                max    min      max   min  ...          max   
country_live                                               ...                
Algeria                      Weekly  Daily  VS Code  Atom  ...         11.0   
Argentina                    Weekly  Daily      Vim  Atom  ...         11.0   
Armenia                      Weekly  Daily      Vim  Atom  ...          6.0   
Australia                    Weekly  Daily      Vim  Atom  ...         11.0   
Austria                      Weekly  Daily      Vim  Atom  ...         11.0   

                                                    several_projects  \
              min                                                max   
country_live                                                           
Algeria       1.0  Yes, I work on one main and several side projects   
Argentina     1.0  Yes, I work on one main and several side projects   
Armenia       1.0  Yes, I work on one main and several side projects   
Australia     1.0  Yes, I work on one main and several side projects   
Austria       1.0  Yes, I work on one main and several side projects   

                                              team_size      use_python_most  \
                                          min       max min              max   
country_live                                                                   
Algeria       No,  I only work on one project        13   1  Web development   
Argentina     No,  I only work on one project        21   1  Web development   
Armenia       No,  I only work on one project        13   1  Web development   
Australia     No,  I only work on one project        41   1  Web development   
Austria       No,  I only work on one project        13   1  Web development   

                                                                 \
                                                            min   
country_live                                                      
Algeria                                           Data analysis   
Argentina                                         Data analysis   
Armenia       DevOps / System administration / Writing autom...   
Australia                                     Computer graphics   
Austria                                       Computer graphics   

             years_of_coding       
                         max  min  
country_live                       
Algeria                 11.0  1.0  
Argentina               11.0  1.0  
Armenia                 11.0  1.0  
Australia               11.0  1.0  
Austria                 11.0  1.0  

[5 rows x 32 columns]

In [33]:
# groupby
(jb2
 .groupby('country_live')
 .agg([min, max])
 .head())

C:\Users\Joanna\AppData\Local\Temp\ipykernel_11696\2558058665.py:4: FutureWarning: ['are_you_datascientist', 'employment_status', 'is_python_main'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  .agg([min, max])


age     company_size        \
             min max          min   max   
country_live                              
Algeria       18  60            1  5000   
Argentina     18  60            1  5000   
Armenia       18  30            1  5000   
Australia     18  60            1  5000   
Austria       18  50            1  5000   

                             first_learn_about_main_ide  \
                                                    min   
country_live                                              
Algeria        Online learning platform / Online course   
Argentina      Online learning platform / Online course   
Armenia        Online learning platform / Online course   
Australia      Online learning platform / Online course   
Austria        Online learning platform / Online course   

                                              how_often_use_main_ide          \
                                          max                    min     max   
country_live                                                                   
Algeria       Technical review / Forum / Blog                  Daily  Weekly   
Argentina     Technical review / Forum / Blog                  Daily  Weekly   
Armenia       Technical review / Forum / Blog                  Daily  Weekly   
Australia     Technical review / Forum / Blog                  Daily  Weekly   
Austria       Technical review / Forum / Blog                  Daily  Weekly   

             ide_main           ...                 several_projects  \
                  min      max  ...                              min   
country_live                    ...                                    
Algeria          Atom  VS Code  ...  No,  I only work on one project   
Argentina        Atom      Vim  ...  No,  I only work on one project   
Armenia          Atom      Vim  ...  No,  I only work on one project   
Australia        Atom      Vim  ...  No,  I only work on one project   
Austria          Atom      Vim  ...  No,  I only work on one project   

                                                                team_size      \
                                                            max       min max   
country_live                                                                    
Algeria       Yes, I work on one main and several side projects         1  13   
Argentina     Yes, I work on one main and several side projects         1  21   
Armenia       Yes, I work on one main and several side projects         1  13   
Australia     Yes, I work on one main and several side projects         1  41   
Austria       Yes, I work on one main and several side projects         1  13   

                                                use_python_most  \
                                                            min   
country_live                                                      
Algeria                                           Data analysis   
Argentina                                         Data analysis   
Armenia       DevOps / System administration / Writing autom...   
Australia                                     Computer graphics   
Austria                                       Computer graphics   

                              years_of_coding       python3_ver       
                          max             min   max         min  max  
country_live                                                          
Algeria       Web development             1.0  11.0         3.5  3.9  
Argentina     Web development             1.0  11.0         3.6  3.9  
Armenia       Web development             1.0  11.0         3.6  3.9  
Australia     Web development             1.0  11.0         3.5  3.9  
Austria       Web development             1.0  11.0         3.5  3.9  

[5 rows x 32 columns]

- What is the minimum and maximum ages and the average team size for each country?

In [35]:
# pivot table
(jb2
 .pivot_table(index='country_live',
              aggfunc={'age': ['min', 'max'],
                       'team_size': 'mean'})
)

age     team_size
              max min      mean
country_live                   
Algeria        60  18  3.722222
Argentina      60  18  4.146789
Armenia        30  18  4.235294
Australia      60  18  3.354015
Austria        50  18  3.132812
...            ..  ..       ...
United States  60  18  4.072673
Uruguay        40  21  3.700000
Uzbekistan     21  21  2.750000
Venezuela      50  18  3.227273
Viet Nam       60  18  4.666667

[76 rows x 3 columns]

In [37]:
# groupby
(jb2
 .groupby('country_live')
 .agg(age_min=('age', min),
      age_max=('age', max),
      team_size_mean = ('team_size', 'mean'))
)

,age_min,age_max,team_size_mean
country_live,,,
Algeria,18,60,3.722222
Argentina,18,60,4.146789
Armenia,18,30,4.235294
Australia,18,60,3.354015
Austria,18,50,3.132812
...,...,...,...
United States,18,60,4.072673
Uruguay,21,40,3.700000
Uzbekistan,21,21,2.750000


## 27.5 Grouping by Hierarchy

- What is the minimum and maximum age for each country and editor?

In [38]:
# pivot table
(jb2
 .pivot_table(index=['country_live', 'ide_main'],
              values='age',
              aggfunc=[min, max])
)

min max
                                          age age
country_live ide_main                            
Algeria      Atom                          21  60
             Eclipse + Pydev               18  18
             IDLE                          40  40
             Jupyter Notebook              30  30
             Other                         30  30
...                                        ..  ..
Viet Nam     Other                         21  21
             PyCharm Community Edition     21  30
             PyCharm Professional Edition  21  21
             VS Code                       18  30
             Vim                           21  40

[813 rows x 2 columns]

In [44]:
# groupby
(jb2
 .groupby(by=['country_live', 'ide_main'])
 [['age']]
 .agg(age_min=('age', min),
      age_max=('age', max))
 )

age_min  age_max
country_live ide_main                                               
Algeria      Atom                                        21       60
             Eclipse + Pydev                             18       18
             Emacs                                     <NA>     <NA>
             IDLE                                        40       40
             IntelliJ IDEA                             <NA>     <NA>
...                                                     ...      ...
Viet Nam     Python Tools for Visual Studio (PTVS)     <NA>     <NA>
             Spyder                                    <NA>     <NA>
             Sublime Text                              <NA>     <NA>
             VS Code                                     18       30
             Vim                                         21       40

[1216 rows x 2 columns]

In [45]:
# drop na
(jb2
 .groupby(by=['country_live', 'ide_main'], observed=True)
 [['age']]
 .agg(age_min=('age', min),
      age_max=('age', max))
 )

age_min  age_max
country_live       ide_main                                   
India              Atom                            18       40
                   Eclipse + Pydev                 18       40
                   Emacs                           21       40
                   IDLE                            18       40
                   IntelliJ IDEA                   21       30
...                                               ...      ...
Dominican Republic Vim                             21       21
Morocco            Jupyter Notebook                30       30
                   PyCharm Community Edition       21       40
                   Sublime Text                    21       30
                   VS Code                         21       30

[813 rows x 2 columns]

## 27.6 Grouping with Functions

In [46]:
def even_grouper(idx):
    return 'odd' if idx % 2 else 'even' # False==1 when remainder present

In [48]:
jb2.pivot_table(index=even_grouper, aggfunc='size')

even    6849
odd     6862
dtype: int64

In [49]:
(jb2
 .groupby(even_grouper)
 .size()
)

even    6849
odd     6862
dtype: int64